In [7]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
K.set_image_dim_ordering('th')
import pickle

In [2]:
num_classes = bird_audio_with_label = pickle.load( open( "cat.pkl", "rb" ) )
X_bird = bird_audio_with_label["cat"]
Y_bird = np.array([[1,0]] * X_bird.shape[0])
print X_bird.shape
print Y_bird.shape

bed_audio_with_label = pickle.load( open( "dog.pkl", "rb" ) )
X_bed = bed_audio_with_label["dog"]
Y_bed = np.array([[0,1]] * X_bed.shape[0])
print X_bed.shape
print Y_bed.shape

X = np.vstack((X_bird, X_bed))
Y = np.vstack((Y_bird, Y_bed))
print X.shape
print Y.shape

(1733, 20, 50)
(1733, 2)
(1746, 20, 50)
(1746, 2)
(3479, 20, 50)
(3479, 2)


In [3]:
X = X.reshape(X.shape[0], 1, 20, 50)
print X.shape
print Y.shape
num_classes = Y.shape[1]
print num_classes

(3479, 1, 20, 50)
(3479, 2)
2


In [4]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(1, 20, 50), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 16, 46)        780       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 8, 23)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 6, 21)         4065      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 3, 10)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 3, 10)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 450)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               57728     
__________

In [8]:
file_path="CNN_val_loss_cat_dog.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
callbacks_list = [checkpoint, early] #early
model.fit(X, Y, validation_split=0.1, callbacks=callbacks_list, epochs=20, batch_size= 100)

Train on 3131 samples, validate on 348 samples
Epoch 1/20
3131/3131 [==============================] - 10s - loss: 1.2878 - acc: 0.6487 - val_loss: 0.4955 - val_acc: 0.7672
Epoch 2/20
3131/3131 [==============================] - 10s - loss: 0.3303 - acc: 0.8579 - val_loss: 0.1075 - val_acc: 0.9655
Epoch 3/20
3131/3131 [==============================] - 10s - loss: 0.1584 - acc: 0.9403 - val_loss: 0.1881 - val_acc: 0.9310
Epoch 4/20
3131/3131 [==============================] - 10s - loss: 0.1057 - acc: 0.9604 - val_loss: 0.1712 - val_acc: 0.9339
Epoch 5/20
3131/3131 [==============================] - 10s - loss: 0.0845 - acc: 0.9681 - val_loss: 0.0434 - val_acc: 0.9828
Epoch 6/20
3131/3131 [==============================] - 10s - loss: 0.0640 - acc: 0.9738 - val_loss: 0.0941 - val_acc: 0.9540
Epoch 7/20
3131/3131 [==============================] - 10s - loss: 0.0630 - acc: 0.9744 - val_loss: 0.0719 - val_acc: 0.9684
Epoch 8/20
3131/3131 [==============================] - 10s - loss: 0.0

In [9]:
model = load_model(file_path)

In [10]:
predicted = model.predict(X)

In [11]:
from sklearn import metrics

In [12]:
metrics.log_loss(Y, predicted)

0.032620967768485762